# Instalar y Cargar bases de datos

In [ ]:
%pip install rdt 
# https://docs.sdv.dev/rdt, esta es donde tome la informacion de para que me servia este libreria

# La librería rdt (Reversible Data Transforms) permite transformar automáticamente columnas de un DataFrame,
# detectando su tipo (numérico, categórico, booleano, fecha, etc.) y aplicando la codificación o normalización más adecuada.
# Es útil para preparar los datos antes de entrenar un modelo, especialmente cuando hay muchas variables de distinto tipo.


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [8]:
import pandas as pd
from rdt import HyperTransformer
import joblib

In [9]:
path = '../data/'
spotify_df_ = pd.read_csv(path + 'spotify-2023-limpio.csv', sep=',', encoding='latin-1') # cargamos el dataset limpio


In [10]:
spotify_df_

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
812,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
813,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
814,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


In [ ]:
# 2️⃣ Seleccionar algunas columnas para la prueba
df = spotify_df_.copy() # Se hace una copia del DataFrame original para no modificarlo directamente

# 4️⃣ Separar variable objetivo
target_column = "streams" # Definimos la variable objetivo
X = df.drop(columns=[target_column]) # Definimos la variable objetivo
y = df[target_column]  # y: variable dependiente

# 5️⃣ Crear el transformador RDT 
ht = HyperTransformer() 

# 6️⃣ Detectar configuración automáticamente ✅
ht.detect_initial_config(X) # El transformador analiza X y determina qué tipo de dato hay en cada columna (numérico, categórico, booleano, etc.)
# Luego asigna automáticamente la transformación más adecuada

# 7️⃣ Ajustar y transformar los datos 🔄
ht.fit(X)   # Aprende cómo transformar cada columna según su tipo
X_transformed = ht.transform(X)  # Aplica las transformaciones aprendidas

# 8️⃣ Convertir a DataFrame para visualizar 🔍
X_transformed = pd.DataFrame(X_transformed, columns=X.columns) 
# Esto se hace para poder visualizarlo y manipularlo fácilmente como DataFrame

# 9️⃣ Mostrar comparación antes y después de la transformación
print("🔹 Datos originales:")
X.head() # Solo imprime los primeros datos antes de la transformación para comparar


🔹 Datos originales:


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,in_apple_playlists,in_apple_charts,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,43,263,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,48,126,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,94,207,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,116,207,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,84,133,...,144,A,Minor,65,23,80,14,63,11,6


In [12]:
print("\n🔹 Datos transformados:")
X_transformed.head()


🔹 Datos transformados:


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,in_apple_playlists,in_apple_charts,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,0.001198,0.000825,2.0,2023.0,7.0,14.0,553.0,147.0,43.0,263.0,...,125.0,0.094240,0.285030,80.0,89.0,83.0,31.0,0.0,8.0,4.0
1,0.001746,0.001548,1.0,2023.0,3.0,23.0,1474.0,48.0,48.0,126.0,...,92.0,0.129582,0.545851,71.0,61.0,74.0,7.0,0.0,10.0,4.0
2,0.003001,0.007827,1.0,2023.0,6.0,30.0,1397.0,113.0,94.0,207.0,...,138.0,0.303729,0.048766,51.0,32.0,53.0,17.0,0.0,31.0,6.0
3,0.004570,0.015737,1.0,2019.0,8.0,23.0,7858.0,100.0,116.0,207.0,...,170.0,0.391818,0.048397,55.0,58.0,72.0,11.0,0.0,11.0,15.0
4,0.005293,0.045818,1.0,2023.0,5.0,18.0,3133.0,50.0,84.0,133.0,...,144.0,0.392951,0.798256,65.0,23.0,80.0,14.0,63.0,11.0,6.0


In [13]:
X_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            816 non-null    float64
 1   artist(s)_name        816 non-null    float64
 2   artist_count          816 non-null    float64
 3   released_year         816 non-null    float64
 4   released_month        816 non-null    float64
 5   released_day          816 non-null    float64
 6   in_spotify_playlists  816 non-null    float64
 7   in_spotify_charts     816 non-null    float64
 8   in_apple_playlists    816 non-null    float64
 9   in_apple_charts       816 non-null    float64
 10  in_deezer_playlists   816 non-null    float64
 11  in_deezer_charts      816 non-null    float64
 12  in_shazam_charts      816 non-null    float64
 13  bpm                   816 non-null    float64
 14  key                   816 non-null    float64
 15  mode                  8

In [ ]:
# 9️⃣ Guardar el dataset limpio y transformado
X_transformed.to_csv(path + 'spotify-2023-limpio-rdt.csv', sep=',', encoding='latin-1', index=False) # guardamos el dataset limpio y transformado

# Guardar
joblib.dump(ht, path + "spotify_transformer.pkl")
# 10️⃣ Guardar el transformador para uso futuro 💾
# Cargar
# ht_ = joblib.load(path + "spotify_transformer.pkl")

# almacenar la etiqueta de la variable objetivo
y.to_csv(path + 'spotify-2023-limpio-rdt-y.csv', sep=',', encoding='latin-1', index=False) # guardamos la variable objetivo